In [2]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split

In [3]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'

train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [4]:
train_input = np.stack((train_q1_data, train_q1_data), axis=1)

In [5]:
print(train_q1_data.shape, train_q2_data.shape)

(298526, 31) (298526, 31)


In [6]:
print(train_input.shape)

(298526, 2, 31)


In [7]:
print(train_input[0])

[[  107    47  1308     4    13     5    27    18   149   207     8  8896
    107    24     5 12288    19     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [  107    47  1308     4    13     5    27    18   149   207     8  8896
    107    24     5 12288    19     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]]


In [8]:
train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)

In [9]:
import xgboost as xgb

In [10]:
train_data = xgb.DMatrix(train_input.sum(axis=1), label=train_label)
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label)

data_list = [(train_data, 'train'), (eval_data, 'valid')]

In [11]:
print(data_list[0])

(<xgboost.core.DMatrix object at 0x000001F0F6509CD0>, 'train')


In [13]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'rmse' #root mean square error

bst = xgb.train(params, train_data, num_boost_round=1000, evals=data_list, early_stopping_rounds=10)

[0]	train-rmse:0.48759	valid-rmse:0.48789
[1]	train-rmse:0.47963	valid-rmse:0.48004
[2]	train-rmse:0.47341	valid-rmse:0.47393
[3]	train-rmse:0.46900	valid-rmse:0.46958
[4]	train-rmse:0.46551	valid-rmse:0.46621
[5]	train-rmse:0.46264	valid-rmse:0.46347
[6]	train-rmse:0.46075	valid-rmse:0.46160
[7]	train-rmse:0.45869	valid-rmse:0.45953
[8]	train-rmse:0.45685	valid-rmse:0.45783
[9]	train-rmse:0.45546	valid-rmse:0.45652
[10]	train-rmse:0.45426	valid-rmse:0.45532
[11]	train-rmse:0.45316	valid-rmse:0.45425
[12]	train-rmse:0.45273	valid-rmse:0.45383
[13]	train-rmse:0.45194	valid-rmse:0.45301
[14]	train-rmse:0.45137	valid-rmse:0.45252
[15]	train-rmse:0.45075	valid-rmse:0.45190
[16]	train-rmse:0.45049	valid-rmse:0.45163
[17]	train-rmse:0.44990	valid-rmse:0.45113
[18]	train-rmse:0.44946	valid-rmse:0.45072
[19]	train-rmse:0.44914	valid-rmse:0.45034
[20]	train-rmse:0.44878	valid-rmse:0.45005
[21]	train-rmse:0.44739	valid-rmse:0.44891
[22]	train-rmse:0.44713	valid-rmse:0.44868
[23]	train-rmse:0.446

In [14]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'), allow_pickle=True)

In [15]:
test_input = np.stack((test_q1_data, test_q2_data), axis=1)
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

In [16]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
output.to_csv(DATA_OUT_PATH + 'simple_xgb.csv', index=False)